In [1]:
import numpy as np
import pandas as pd
import time
import ast
from tqdm import tqdm
import datetime
from multiprocessing import Pool, cpu_count
from itertools import zip_longest
from collections import defaultdict, OrderedDict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from joblib import Parallel, delayed
from datetime import datetime 
%matplotlib inline

In [2]:
base_dir = './dataset/'

### app使用日志文件

In [3]:
user_app_usage=pd.read_csv(base_dir+'user_app_usage.csv',header=None,names=['uId','appId','duration','times','use_date'],
                           dtype={'uId':int,'duration':np.uint32,'times':np.uint32,'use_date':'category'})
user_app_usage.shape

(840560515, 5)

In [4]:
user_app_usage.head(10)

,uId,appId,duration,times,use_date
0,20765863,a003838,3012,67,2019-07-16
1,20765863,a003838,1670,11,2019-06-28
2,20765863,a003838,390,29,2019-07-08
3,20765863,a003838,1793,12,2019-07-24
4,20765863,a003838,6905,33,2019-07-06
5,20765863,a003838,47,6,2019-07-25
6,20765863,a003838,2557,39,2019-07-05
7,20765863,a003838,3730,43,2019-07-12
8,20765863,a003838,6905,33,2019-07-07
9,20765863,a003838,266,19,2019-07-11


In [5]:
app_usage_count_df = pd.read_csv(base_dir+'pd_user_app_usage_appid_count.csv')
app_usage_count_df.shape

(9353, 2)

In [6]:
app_usage_top_100_list = list(app_usage_count_df.loc[:99, 'appId'])

In [7]:
user_app_usage['is_hot'] = user_app_usage['appId'].apply(lambda x:1 if x in app_usage_top_100_list else 0)

In [8]:
user_app_usage_hot = user_app_usage[user_app_usage['is_hot']==1]
user_app_usage_hot.shape

(657067683, 6)

In [9]:
user_app_usage_hot['idx'] = user_app_usage_hot['appId'].apply(lambda x:app_usage_top_100_list.index(x))
user_app_usage_hot.drop(['is_hot'],axis=1, inplace=True)
user_app_usage_hot['idx'] = user_app_usage_hot['idx'].astype(np.uint8)
user_app_usage_hot.drop(['appId'],axis=1, inplace=True)
user_app_usage_hot['uId'] = user_app_usage_hot['uId'].astype(np.uint32)
user_app_usage_hot['use_date'] = user_app_usage_hot['use_date'].astype(str)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [10]:
user_app_usage_hot.head(10)

,uId,duration,times,use_date,idx
30,20765863,701,2,2019-07-14,30
31,20765863,51,3,2019-07-23,30
32,20765863,701,2,2019-07-13,30
33,20765863,2819,2,2019-07-05,30
34,20765863,2819,2,2019-07-06,30
38,20765863,285,5,2019-07-23,68
39,20765863,285,5,2019-07-24,68
40,20765863,305,7,2019-07-20,68
41,20765863,6705,172,2019-07-24,0
42,20765863,2249,93,2019-07-03,0


In [11]:
user_app_usage_hot['idx'].max()

99

In [5]:
user_app_usage_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443257438 entries, 0 to 651007718
Data columns (total 5 columns):
uId         uint32
duration    uint32
times       uint32
use_date    object
idx         uint8
dtypes: object(1), uint32(3), uint8(1)
memory usage: 12.0+ GB


In [12]:
user_app_usage_hot['use_date'] = pd.to_datetime(user_app_usage_hot['use_date'])
user_app_usage_hot['dayofweek'] = user_app_usage_hot['use_date'].dt.dayofweek
user_app_usage_hot['dayofweek'] = user_app_usage_hot['dayofweek'].astype(np.uint8)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [13]:
user_app_usage_hot['duration'] = user_app_usage_hot['duration'].astype(np.uint16)
user_app_usage_hot['times'] = user_app_usage_hot['times'].astype(np.uint16)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
h5 = pd.HDFStore(base_dir+'user_app_usage_hot_100.h5','w',complevel=4,complib='blosc')
h5['data'] = user_app_usage_hot
h5.close()

In [3]:
user_app_usage_hot = pd.read_hdf(base_dir+'user_app_usage_hot_100.h5',key='data')
user_app_usage_hot.head()

,uId,duration,times,use_date,idx,dayofweek
30,20765863,701,2,2019-07-14,30,6
31,20765863,51,3,2019-07-23,30,1
32,20765863,701,2,2019-07-13,30,5
33,20765863,2819,2,2019-07-05,30,4
34,20765863,2819,2,2019-07-06,30,5


In [6]:
# def build_usage_app_table(group_df):
#     (uId, use_date),group = group_df
#     app_times = np.zeros((50,),dtype=np.int16)
#     app_durations = np.zeros((50,),dtype=np.int16)
#     idx = list(group['idx'])
#     times = list(group['times'])
#     durations = list(group['duration'])
#     app_times[idx] = times
#     app_durations[idx] = durations
#     app_usage = np.concatenate([app_times, app_durations]).reshape((1,-1))
#     app_usage_df = pd.DataFrame(app_usage)
#     app_usage_df['uId'] = uId
#     app_usage_df['use_date'] = use_date
#     return app_usage_df

In [4]:
def build_usage_app_table_by_dayofweek(group_df):
    (uId, dayofweek),group = group_df
    app_times = np.zeros((100,),dtype=np.int16)
    app_durations = np.zeros((100,),dtype=np.int16)
    idx = list(group['idx'])
    times = list(group['times'])
    durations = list(group['duration'])
    for i in range(len(idx)):
        app_times[idx[i]] += times[i]
        app_durations[idx[i]] += durations[i]
    app_usage = np.concatenate([app_times, app_durations]).reshape((1,-1))
    app_usage_df = pd.DataFrame(app_usage)
    app_usage_df['uId'] = uId
    app_usage_df['dayofweek'] = dayofweek
    return app_usage_df

In [5]:
len(user_app_usage_hot['uId'].unique())

4011684

In [6]:
all_uIds = list(user_app_usage_hot['uId'].unique())

In [7]:
uId_size = 700000
uId_list = []
for i in range(0, len(all_uIds), uId_size):
    uId_list.append(all_uIds[i:i+uId_size])

In [8]:
uId_array = np.array(uId_list)
uId_array.shape

(6,)

In [9]:
np.save('./uId_array.npy',uId_array)

In [10]:
uIds = np.load('./uId_array.npy')
len(uIds[0])

700000

In [ ]:
###- ----------------------------------------

In [11]:
uId_df = pd.DataFrame(uIds[0])
uId_df.shape

(700000, 1)

In [12]:
uId_df.columns = ['uId']

In [13]:
uId_df.head()

,uId
0,20765863
1,20768838
2,20776586
3,20780565
4,20781195


In [14]:
sub_user_app_usage_hot = uId_df.merge(user_app_usage_hot,on=['uId'],how='left')
sub_user_app_usage_hot.shape

(114536812, 6)

In [25]:
sub_user_app_usage_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44719036 entries, 0 to 44719035
Data columns (total 6 columns):
uId          uint32
duration     uint16
times        uint16
use_date     datetime64[ns]
idx          uint8
dayofweek    uint8
dtypes: datetime64[ns](1), uint16(2), uint32(1), uint8(2)
memory usage: 1.1 GB


In [15]:
sub_user_app_usage_hot['uId'] = sub_user_app_usage_hot['uId'].astype(np.uint32)

In [16]:
df_list = []
i = 0
for (uId, dayofweek),group in sub_user_app_usage_hot.groupby(['uId','dayofweek']):
    a = build_usage_app_table_by_dayofweek(((uId,dayofweek),group))
    df_list.append(a)
    i += 1
    if i % 1000000 == 0:
        print(i)

1000000
2000000
3000000
4000000


In [17]:
top_100_app_usage_df = pd.concat(df_list)
top_100_app_usage_df.shape

(4767423, 202)

In [18]:
top_100_app_usage_df.head()

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,uId,dayofweek
0,1256,113,0,0,296,0,0,0,33,0,...,0,0,0,0,0,0,0,0,1000057,0
0,1227,346,0,0,290,8,0,4,0,0,...,0,0,0,0,0,0,0,0,1000057,1
0,1398,279,3,0,225,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000057,2
0,1341,204,0,0,270,0,0,10,0,0,...,0,0,0,0,0,0,0,0,1000057,3
0,1155,198,0,0,362,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000057,4


In [19]:
h5 = pd.HDFStore(base_dir+'top_100_app_usage_df_0.h5','w')
h5['data'] = top_100_app_usage_df
h5.close()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  


In [40]:
# a = pd.read_hdf('./top_100_app_usage_df_0.h5',key='data')
# a.head()

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,uId,use_date
0,159,8,4,0,0,4,6,0,0,0,...,0,0,0,0,0,0,0,0,1000003,2019-02-28
0,101,11,23,0,0,4,2,0,0,0,...,0,0,0,0,0,0,0,0,1000003,2019-03-01
0,128,21,67,0,0,23,2,0,0,0,...,0,0,0,0,0,0,0,0,1000003,2019-03-02
0,119,17,9,0,19,44,5,0,18,0,...,0,0,0,0,0,0,0,0,1000003,2019-03-03
0,205,59,39,0,19,30,5,0,10,0,...,0,0,0,0,0,0,0,0,1000003,2019-03-04


In [41]:
a.shape

(5175388, 202)

### 构造全部用户30天数据

In [3]:
# uId_list = []
# for uId in uIds:
#     uId_list.extend(list(uId))
# len(uId_list)

1985572

In [3]:
train = pd.read_csv(base_dir+'age_train.csv',names=['uId','age_group'],dtype={'uId':np.uint32,'age_group':np.uint8})
train.shape

(4000000, 2)

In [17]:
train['age_group'].value_counts()

4    800000
3    800000
5    700000
2    700000
6    500000
1    500000
Name: age_group, dtype: int64

In [4]:
test = pd.read_csv(base_dir+'age_test.csv',names=['uId'],dtype={'uId':np.uint32})
test.shape

(1000000, 1)

In [5]:
uId_list = pd.concat([train['uId'],test['uId']])
uId_list.shape

(5000000,)

In [6]:
uId_list = list(uId_list)
len(uId_list)

5000000

In [7]:
uId_df = pd.DataFrame(uId_list)
uId_df.columns = ['uId']
uId_df.shape

(5000000, 1)

In [8]:
# def datelist(beginDate, endDate): # beginDate, endDate是形如‘20190228’的字符串或datetime格式 
#     date_l=[datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start=beginDate, end=endDate))] 
#     return date_l

In [9]:
# d_l = datelist('20190228','20190329')
# d_l

['2019-02-28',
 '2019-03-01',
 '2019-03-02',
 '2019-03-03',
 '2019-03-04',
 '2019-03-05',
 '2019-03-06',
 '2019-03-07',
 '2019-03-08',
 '2019-03-09',
 '2019-03-10',
 '2019-03-11',
 '2019-03-12',
 '2019-03-13',
 '2019-03-14',
 '2019-03-15',
 '2019-03-16',
 '2019-03-17',
 '2019-03-18',
 '2019-03-19',
 '2019-03-20',
 '2019-03-21',
 '2019-03-22',
 '2019-03-23',
 '2019-03-24',
 '2019-03-25',
 '2019-03-26',
 '2019-03-27',
 '2019-03-28',
 '2019-03-29']

In [10]:
# uId_df['date_list'] = uId_df.apply(lambda x:d_l,axis=1)

In [11]:
# uId_df.head()

,uId,date_list
0,1000001,"[2019-02-28, 2019-03-01, 2019-03-02, 2019-03-0..."
1,1000011,"[2019-02-28, 2019-03-01, 2019-03-02, 2019-03-0..."
2,1000015,"[2019-02-28, 2019-03-01, 2019-03-02, 2019-03-0..."
3,1000019,"[2019-02-28, 2019-03-01, 2019-03-02, 2019-03-0..."
4,1000023,"[2019-02-28, 2019-03-01, 2019-03-02, 2019-03-0..."


In [12]:
# uId_df=pd.DataFrame({'uId':uId_df.uId.repeat(uId_df.date_list.str.len()),'date':np.concatenate(uId_df.date_list.values)})
# uId_df.shape

(75375000, 2)

In [13]:
# uId_df.head()

,date,uId
0,2019-02-28,1000001
0,2019-03-01,1000001
0,2019-03-02,1000001
0,2019-03-03,1000001
0,2019-03-04,1000001


In [14]:
# uId_df.rename(columns={'date':'use_date'},inplace=True)

In [15]:
# uId_df['uId'] = uId_df['uId'].astype(np.uint32)

In [16]:
# uId_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75375000 entries, 0 to 2512499
Data columns (total 2 columns):
use_date    object
uId         uint32
dtypes: object(1), uint32(1)
memory usage: 1.4+ GB


### 构造每周

In [8]:
def daylist():
    day_list = list(range(7))
    return day_list

In [9]:
day_list = daylist()
day_list

[0, 1, 2, 3, 4, 5, 6]

In [10]:
uId_df['date_list'] = uId_df.apply(lambda x:day_list,axis=1)
uId_df.head()

,uId,date_list
0,32689591,"[0, 1, 2, 3, 4, 5, 6]"
1,32689586,"[0, 1, 2, 3, 4, 5, 6]"
2,32689496,"[0, 1, 2, 3, 4, 5, 6]"
3,32689481,"[0, 1, 2, 3, 4, 5, 6]"
4,32689295,"[0, 1, 2, 3, 4, 5, 6]"


In [11]:
uId_df=pd.DataFrame({'uId':uId_df.uId.repeat(uId_df.date_list.str.len()),'day':np.concatenate(uId_df.date_list.values)})
uId_df.shape

(35000000, 2)

In [18]:
uId_df.head()

,uId,day
0,32689591,0
0,32689591,1
0,32689591,2
0,32689591,3
0,32689591,4


In [19]:
uId_df.rename(columns={'day':'dayofweek'},inplace=True)
uId_df['uId'] = uId_df['uId'].astype(np.uint32)

In [20]:
df_dirs = [base_dir+'top_100_app_usage_df_{}.h5'.format(i) for i in range(6)]
df_dirs

['./dataset/top_100_app_usage_df_0.h5',
 './dataset/top_100_app_usage_df_1.h5',
 './dataset/top_100_app_usage_df_2.h5',
 './dataset/top_100_app_usage_df_3.h5',
 './dataset/top_100_app_usage_df_4.h5',
 './dataset/top_100_app_usage_df_5.h5']

In [21]:
df_list = []
for df_dir in df_dirs:
    df = pd.read_hdf(df_dir,key='data')
    df_list.append(df)

In [22]:
top_100_app_usage_df = pd.concat(df_list)
top_100_app_usage_df.shape

(27325718, 202)

In [23]:
top_100_app_usage_df.head()

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,uId,dayofweek
0,1256,113,0,0,296,0,0,0,33,0,...,0,0,0,0,0,0,0,0,1000057,0
0,1227,346,0,0,290,8,0,4,0,0,...,0,0,0,0,0,0,0,0,1000057,1
0,1398,279,3,0,225,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000057,2
0,1341,204,0,0,270,0,0,10,0,0,...,0,0,0,0,0,0,0,0,1000057,3
0,1155,198,0,0,362,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000057,4


In [24]:
len(top_100_app_usage_df['uId'].unique())

4011684

In [25]:
len(top_100_app_usage_df['dayofweek'].unique())

7

In [26]:
uId_df = uId_df.merge(top_100_app_usage_df,on=['uId','dayofweek'],how='left')
uId_df.shape

(35000000, 202)

In [27]:
uId_df.head(10)

,uId,dayofweek,0,1,2,3,4,5,6,7,...,190,191,192,193,194,195,196,197,198,199
0,32689591,0,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,32689591,1,436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32689591,2,382.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32689591,3,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32689591,4,609.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,32689591,5,457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,32689591,6,364.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,32689586,0,943.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,32689586,1,738.0,0.0,19.0,0.0,49.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,32689586,2,958.0,0.0,19.0,0.0,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
uId_df.fillna(0, inplace=True)

In [29]:
h5 = pd.HDFStore(base_dir+'top_100_app_usage_30d_df.h5','w',complevel=4,complib='blosc')
h5['data'] = uId_df
h5.close()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  


In [37]:
print(0)

0


In [6]:
# uId_df = pd.read_hdf(base_dir+'top_100_app_usage_30d_df.h5',key='data')
# uId_df.shape

(35000000, 402)

In [7]:
# uId_df.head()

,uId,dayofweek,0,1,2,3,4,5,6,7,...,390,391,392,393,394,395,396,397,398,399
0,32689591,0,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,32689591,1,436.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32689591,2,382.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32689591,3,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32689591,4,609.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
uId_df.drop(['dayofweek'],axis=1,inplace=True)

In [31]:
train.head()

,uId,age_group
0,32689591,6
1,32689586,6
2,32689496,6
3,32689481,6
4,32689295,6


In [32]:
test.head()

,uId
0,6708326
1,6708314
2,6708313
3,6708284
4,6708272


In [33]:
train_lstm_input = train[['uId']].merge(uId_df,on=['uId'],how='left')
train_lstm_input.drop(['uId'],inplace=True,axis=1)
train_lstm_input.shape

(28000000, 200)

In [34]:
lstm_scaler = StandardScaler().fit(train_lstm_input)
train_lstm_input = lstm_scaler.transform(train_lstm_input)
train_lstm_input.shape

(28000000, 200)

In [38]:
h5 = pd.HDFStore(base_dir+'train_lstm_input.h5','w',complevel=4,complib='blosc')
h5['data'] = pd.DataFrame(train_lstm_input)
h5.close()

In [41]:
test_lstm_input = test[['uId']].merge(uId_df,on=['uId'],how='left')
test_lstm_input.drop(['uId'],inplace=True,axis=1)
test_lstm_input.shape

(7000000, 200)

In [42]:
test_lstm_input = lstm_scaler.transform(test_lstm_input)

In [43]:
h5 = pd.HDFStore(base_dir+'test_lstm_input.h5','w',complevel=4,complib='blosc')
h5['data'] = pd.DataFrame(test_lstm_input)
h5.close()